In [1]:
from torch_geometric.datasets import FacebookPagePage

dataset = FacebookPagePage(root="./data/Facebook/")
data = dataset[0]

/Users/air/Desktop/GNN/venv/lib/python3.10/site-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load(f

In [2]:
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')

Number of features: 128
Number of classes: 4
Number of nodes: 22470
Number of edges: 342004


In [3]:
data.train_mask = range(18000)
data.val_mask = range(18001, 20000)
data.test_mask = range(20001, 22470)

# experiment 1

Run 4 models for 128 features and 4 classes. 100 epochs of training.
1. Multi Layer Perceptron (128,16,4)


In [ ]:
from MyNN import MLP, GCN, GAT

In [ ]:
mlp = MLP(dataset.num_features, 16, dataset.num_classes)
print(mlp)
mlp.fit(data, epochs=100)
acc = mlp.test(data)
print(f'MLP test accuracy: {acc*100:.2f}%')

In [ ]:
gcn = GCN(dataset.num_features, 16, dataset.num_classes)
print(gcn)
gcn.fit(data, epochs=100)
acc_gcn = gcn.test(data)
print(f'GCN test accuracy: {acc_gcn*100:.2f}%')

In [ ]:
from utils import visualize

In [ ]:
gcn.eval()
out = gcn(data.x, data.edge_index)
visualize(out, color=data.y)

In [ ]:
mlp.eval()
out2 = mlp(data.x)
visualize(out2, color=data.y)

In [ ]:
gat = GAT(dataset.num_features, 16, dataset.num_classes, heads=8)
print(gat)
gat.fit(data, epochs=100)
acc_gat = gat.test(data)
print(f'GAT test accuracy: {acc_gat*100:.2f}%')

In [ ]:
gat.eval()
out3 = gat(data.x,data.edge_index)
visualize(out3, color=data.y)

In [ ]:
from utils import to_torch_data
data2 = to_torch_data('data/edge_facebook.parquet')

In [ ]:
data2


In [ ]:
import pyarrow.parquet as pq

In [ ]:
df_edges = pq.read_table('data/edge_facebook.parquet')
df = pq.read_table('data/facebook.parquet')

In [ ]:
df_e = df_edges.to_pandas()
df = df.to_pandas()

In [ ]:
import numpy as np

In [ ]:
df_e = df_e.apply(lambda x: x-1, axis=1)

df_x = df.drop(columns=['target','node_id']).astype(np.float32)
df_y = df['target'].astype(np.int32)

In [ ]:
import torch
edge_index = torch.from_numpy(df_e[["in","out"]].to_numpy())
x = torch.from_numpy(np.array(df_x))
y = torch.from_numpy(np.array(df_y))
y = y.type(torch.LongTensor)

In [ ]:
from torch_geometric.data import Data
data3 = Data(x=x, edge_index=edge_index.T, y=y)
# for test 
data4 = Data(x=x, edge_index=data2.edge_index, y=y)

In [ ]:
print(f'Number of features: {data3.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

print(f'Number of nodes: {data3.num_nodes}')
print(f'Number of edges: {data3.num_edges}')

In [ ]:
print(f'Number of features: {data4.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

print(f'Number of nodes: {data4.num_nodes}')
print(f'Number of edges: {data4.num_edges}')

In [ ]:
data3.train_mask = range(18000)
data3.val_mask = range(18001, 20000)
data3.test_mask = range(20001, 22470)

data4.train_mask = range(18000)
data4.val_mask = range(18001, 20000)
data4.test_mask = range(20001, 22470)

In [ ]:
from MyNN import MLP
mlp = MLP(data3.num_features, 16, 2)
print(mlp)
mlp.fit(data3, epochs=100)
acc = mlp.test(data3)
print(f'MLP test accuracy: {acc*100:.2f}%')

In [ ]:
from MyNN import MLP
mlp2 = MLP(data4.num_features, 16, 2)
print(mlp)
mlp2.fit(data4, epochs=100)
acc2 = mlp2.test(data4)
print(f'MLP test accuracy: {acc2*100:.2f}%')

In [ ]:
from MyNN import GCN
gcn2 = GCN(data3.num_features, 16, 2)
print(gcn2)
gcn2.fit(data3, epochs=100)
acc_gcn2 = gcn2.test(data3)
print(f'GCN test accuracy: {acc_gcn2*100:.2f}%')

In [ ]:
from MyNN import GCN
gcn3 = GCN(data4.num_features, 16, 2)
print(gcn3)
gcn3.fit(data4, epochs=100)
acc_gcn3 = gcn3.test(data4)
print(f'GCN test accuracy: {acc_gcn3*100:.2f}%')

In [ ]:
from MyNN import GAT
gat = GAT(data3.num_features, 8, 2, heads=8)
print(gat)
gat.fit(data3, epochs=100)
acc_gat = gat.test(data3)
print(f'GAT test accuracy: {acc_gat*100:.2f}%')

In [ ]:
from MyNN import GAT
gat2 = GAT(data4.num_features, 8, 2, heads=8)
print(gat2)
gat2.fit(data4, epochs=100)
acc_gat3 = gat2.test(data4)
print(f'GAT test accuracy: {acc_gat3*100:.2f}%')

In [ ]:
gcn2.eval()
out = gcn2(data3.x, data3.edge_index)
visualize(out, color=data3.y)